In [1]:
# Instalamos el driver de cassandra
# sudo pip install cassandra-driver

In [13]:
# Vamos a importar
from cassandra.cluster import Cluster
# Vamos a conectarno a la base de datos:

## Iniciareos el contenedor de Docker con la iagen de cassandra:
'''
docker pull cassandra

Cluster: Para esta práctica solo usaremos un nodo
docker run --name prac2_a -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

docker run --name prac2 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d imagen-cassandra

docker run --name prac2 -p 192.168.99.100:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d imagen-cassandra

Conexión:
docker exec -it prac2 bash

Información del Nodo:
nodetool status

'''
# Indicación de nuestra imagen

cluster = Cluster(contact_points=['0.0.0.0'], port=9042)

# Generar la conexión
session = cluster.connect()

NoHostAvailable: ('Unable to connect to any servers', {'0.0.0.0:9042': ConnectionShutdown('Connection to 0.0.0.0:9042 was closed')})

In [ ]:
'''
Creación del KEYSPACE:  que vamos a llamar peliculas
y le vamos a poner una estrategia de replicación de tipo 
SimpleStrategy y con un factor de replicación igual a 1.
'''
session.execute("""CREATE KEYSPACE base_datos WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};""")


### Creación de la Tablas.

In [ ]:
# Seleccionamos el KEYSPACE
session.set_keyspace('base_datos')

In [ ]:
# Creamos la tabla
session.execute("""
                CREATE TABLE resenas (
                    membresia int,
                    id_libro int,
                    titulo text,
                    autor text, 
                    categoria text,
                    calificacion float,
                    nombre text STATIC,
                    a_p text STATIC,
                    a_m text STATIC,
                    pais text STATIC,
                    fechnac date STATIC,
                    PRIMARY KEY (membresia, id_libro));
                """)


In [ ]:
'''
Podemos consultar las tablas creadas con:
DESCRIBE tables

Vamos a activar la vista materialziada

'''

# Creamos la vista materializada
session.execute("""
                CREATE MATERIALIZED VIEW top AS SELECT
                membresia, id_libro, categoria, calificacion, titulo, autor
                FROM resenas WHERE membresia IS NOT NULL AND id_libro IS NOT NULL AND
                categoria IS NOT NULL AND calificacion IS NOT NULL AND titulo IS NOT NULL AND 
                autor IS NOT NULL PRIMARY KEY (categoria, id_libro, membresia);
                """)

In [ ]:
'''
Ingresamos los datos:
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61682,5,'100 años de soledad','Gabriel García Marquez','Realismo Mágico',5.00,'Daniel','Fragoso','Alvarado','Mexico','2000-08-16');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,6,'La casa de Bernarda Alba','Federico García Lorca','Autobiografía',5.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61684,7,'Una educación','Tara Westover','Ciencia Ficcion',5.00,'Hugo','Rangel','Ramirez','Mexico','2000-04-06');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,8,'Do androids dream of electric sheep?','Philip K. Dick','Novela',4.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61684,9,'Solo quedó nuestra historia','Adam Silvera','Cuento',3.00,'Hugo','Rangel','Ramirez','Mexico','2000-04-06');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61684,10,'El Tapiz Amarillo','Charlotte Perkins Gilman','Novela',4.00,'Hugo','Gómez','Martinez','Mexico','2000-04-06');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,11,'Lazarillo de Tormes','NULL','Novela',1.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61682,12,'La cancion de Aquiles','Madeline Miller','Novela',5.00,'Daniel','Fragoso','Alvarado','Mexico','2000-08-16');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,13,'Doña Perfecta','Benito Pérez Galdós','Cuento',4.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61684,14,'La metamorfosis','Franz Kafka','Cuento',4.00,'Hugo','Rangel','Ramirez','Mexico','2000-04-06');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,15,'Erecciones, eyaculaciones, exhibiciones','Charles Bukowski','Ciencia Ficcion',4.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,16,'Un mundo feliz','Aldous Huxley','Novela',5.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,17,'Ender''s Game','Orson Scott Card','Novela',3.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61682,18,'Pedro Paramo','Juan Rulfo','Novela',3.00,'Daniel','Fragoso','Alvarado','Mexico','2000-08-16');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,19,'Travesuras de la niña mala','Mario Vargas Llosa','Novela',5.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61684,12,'La cancion de Aquiles','Madeline Miller','Novela',4.00,'Hugo','Rangel','Ramirez','Mexico','2000-04-06');
INSERT INTO resenas(membresia,id_libro,titulo,autor,categoria,calificacion,nombre,a_p,a_m,pais,fechnac) VALUES (61683,20,'El Periquillo de Sarniento','José Joaquín Fernández de Lizardi','Novela',2.00,'Enrique','Gómez','Martinez','Mexico','1999-07-10');

'''